# Import External Dependencies

In [28]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np
import os
import itertools
import io
import tensorflow as tf

from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# visualization dependencies
import cv2
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import iplot, plot
from plotly.offline import init_notebook_mode

import cufflinks as cf
import warnings

warnings.filterwarnings('ignore')

cf.go_offline()
cf.set_config_file(offline=False, world_readable=False)

init_notebook_mode(connected=False)

plt.rcParams['figure.dpi'] = 120.0
plt.rcParams['font.sans-serif'] = 'Lato'
sns.set(style='whitegrid', palette='muted',
              rc={'figure.figsize': (16, 16)})

tf.config.run_functions_eagerly(True)



In [29]:
IMG_SIZE = 256
NUM_EPOCHS = 50
LR = 1e-4
BATCH_SIZE = 32
NUM_CLASSES = 4

In [30]:
import shutil

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
    <script src="/static/components/requirejs/require.js"></script>
    <script>
      requirejs.config({
        path: {
          base: '/static/base',
          plotly: "https://cdn.plot.ly/plotly-1.5.1.min.js?noext",
        },
      });
    </script>
  '''))

def clear_content(directory):
  for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
    except Exception as exc:
      raise("Failed to Delete: %s %s" % (file_path, exc))

# Step 1: Load and Inspect the Dataset

In [36]:
!conda install -y gdown
!gdown --id 1r0k6CsrwWAQKY-pfMPJjvq8f9VOo6wYs
!gdown --id 1tkO_BML_6zEb6wNAGYfX7ro_EtJDyrjG

In [38]:
!gdown --id 1-749mKKw7du1verM4MNsTKXQcGkICEQf
!gdown --id 1mTG7Sj9ouZ49lLJFZTtEMM4Ryal4PIQ6
!gdown --id 1VmPc29YY3ScYySmuPXSIqtd1L2DtvXB-

In [33]:
!gdown --id 1XLClixZesnrPoF_wANWWEPtZZeR1_ZtH
!gdown --id 1zGkHR7lf6fZpuA3G49T3WqeOng_CjYiV # rgb_merged_model
!gdown --id 10k7GKAJIWjcLXPX8V6OfPWHTUWYomDlJ # test images

In [34]:
!pip install openpyxl

In [35]:
from zipfile import ZipFile
from tqdm import tqdm

prep_path = "/kaggle/working/preprocessed_images.zip"
odir_path = "/kaggle/working/ocular-disease-recognition-odir5k.zip"

paths = ['ocular_dataset_jpg.zip','Glaucoma.zip','ocular_dataset_jpg_400.zip','datazet.zip','ocular-disease-recognition-odir5k.zip', 'preprocessed_images.zip', 'odir_rgb_merged_model_four-0.9248.zip']
names = ['ocular_dataset_jpg','Glaucoma','ocular_dataset_jpg_400','datazet','odir5k', 'preprocessed_images', 'odir_rgb_merged_model_four-0.9248']

for i in range(7):
    if os.path.exists(names[i]):
        clear_content(names[i])
    else:
        os.mkdir(names[i])
        
    with ZipFile(paths[i], 'r') as obj:
        file_list = obj.namelist()
        for file in tqdm(file_list):
            obj.extract(file, names[i])
    print()
    print(f"{names[i]} Data extracted!")

In [ ]:
from lib2to3.pygram import Symbols
import random
import numbers

upper = "ABCEFGHIJKLMNOPQRSTUVWXYZ"
lower = "abcdefghijklmnopqrstuvwxyz"
numbers = "0123456789"
symbols = "[]{}()*;/,.@$"

all = lower + symbols + upper + numbers 
length = 16
passcode = "".join(random.sample(all, length))
print(passcode)

In [ ]:
for dirname, subfolders, filenames in os.walk("/kaggle/working/Glaucoma"):
  for subf in subfolders:
    print(os.path.join(dirname, subf))
    
print(len(os.listdir("Glaucoma/Glaucoma")))
# print(len(os.listdir("datazet/datazet/glaucoma")))
# print(len(os.listdir("datazet/datazet/myopia")))
# print(len(os.listdir("datazet/datazet/cataract")))

In [ ]:
train_dir = "odir5k/ODIR-5K/ODIR-5K/Training Images"
test_dir = "odir5k/ODIR-5K/ODIR-5K/Testing Images"
# occ_path = "ocular_dataset_jpg/occ_preprocessed_jpg"
# occ_annot_path = "ocular_dataset_jpg_/ocular_disease_annotation_upsample.csv"
occ_path = "ocular_dataset_jpg_400/occ_preprocessed_jpg"
occ_annot_path = "ocular_dataset_jpg_400/ocular_disease_annotation_upsample.csv"
annotations_path = "odir5k/full_df.csv"
glaucoma_ds = "Glaucoma/Glaucoma"
prep_train = "preprocessed_images/preprocess_train"
prep_test = "preprocessed_images/preprocess_test"
discarded_path = "DiscardedImages.xlsx"
test_folder = "datazet/datazet"
# odir_model_dir = "odir_model_eight-0.9059/trained_models/odir"
# odir_model_dir = "odir_model_eight-0.9059/trained_models/odir/odir_model-0.91"
# odir_dir = "odir_model_eight-0.9059/trained_models/odir"
# odir_model_path = "odir_model_four-0.9407/trained_models/odir/odir_model-0.94"
# odir_rgb_model_path = "RGB_odir_model_four-0.9162/trained_models/odir/odir_model-0.92"
odir_model_merged = "/kaggle/working/odir_rgb_merged_model_four-0.9248/trained_models/odir/odir_model-0.92"
odir_history_merged = "/kaggle/working/odir_rgb_merged_model_four-0.9248/trained_models/odir/odir_history.h5"
files_train = sorted(os.listdir(train_dir))
files_test = sorted(os.listdir(test_dir))
glaucoma_data = sorted(os.listdir(glaucoma_ds))

print("train: " , len(files_train))
print("test: ", len(files_test))
print("occ: ", len(os.listdir(occ_path)))

In [ ]:
folders = ['normal', 'cataract','myopia', 'glaucoma']
def remove_non_img(directory):
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isdir(file_path):
            shutil.rmtree(file_path)
        elif os.path.isfile(file_path):
            if not file_path.endswith('.png') and not file_path.endswith('.jpg'):
                os.unlink(file_path)
            
for n in range(4):
    direc = os.path.join(test_folder, folders[n])
#     print(direc)
    remove_non_img(direc)

In [ ]:
len(os.listdir(prep_train))

### Visualize Retinal Images

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0.1, hspace=0.2)

for i in range(25):
  rand_idx = np.random.randint(0, len(glaucoma_data))
  image = cv2.imread(os.path.join(glaucoma_ds, glaucoma_data[rand_idx]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.subplot(5, 5, i+1)
  plt.imshow(image)
  plt.title(f"{image.shape}")
  plt.axis('off')
  plt.grid(False)
plt.show()

In [ ]:
# pandas filtering functions

def data_mapping(df, initial, odir=True):
  labels = []
  
  if odir:
    fundus_names = list(df.iloc[:, 3])
  elif not odir:
    fundus_names = list(df.iloc[:, 0])

  for i in range(len(fundus_names)):
    labels.append(initial)

  return labels

def count_labels(df, odir=True):
  if odir:
#       labels = list(df.iloc[:, -12:-4].sum().index)
    labels = ["N", "C", "G", "M"]
  elif not odir:
    labels = list(df.iloc[:, 1:5].sum().index)
#   labels = ['N', 'C']
  total_count = []
  labels_count = []

  for i in range(len(labels)):
    label_count = data_mapping(df.loc[(df[labels[i]] == 1)], labels[i], odir)
    total_count += label_count
    labels_count.append(len(label_count))
    print(f"{labels[i]}: {len(label_count)}")

  return total_count, labels_count

In [ ]:
annot_df = pd.read_csv(annotations_path)
occ_df = pd.read_csv(occ_annot_path)
occ_df.info()

In [ ]:
occ_df.iloc[:, 1:5].sum().index

In [ ]:
annot_df.iloc[:, -12:-4].sum().index

In [ ]:
annot_df.head(10)

In [ ]:
discarded_sheet = pd.read_excel(discarded_path, sheet_name="Sheet1")
# annot_df.loc[(annot_df["Left-Fundus"].isin(discarded_sheet["Left-Fundus"]))].drop(discarded_sheet['Left-Fundus'], axis=0)
dropped = annot_df.loc[~annot_df['Left-Fundus'].isin(discarded_sheet['Left-Fundus'])]
print(len(dropped.loc[(dropped['Left-Fundus'].isin(discarded_sheet['Left-Fundus']))]))
print(len(dropped.loc[(dropped['Left-Fundus'].str.match("4601_left.jpg"))]))

# ---- these below are not working ---- #
# duplicates = pd.merge(annot_df['Left-Fundus'], discarded_sheet['Left-Fundus'],
#                       how='inner', left_on=['Left-Fundus'], right_on=['Left-Fundus'],
#                       left_index=True)
# dropped = annot_df.loc[(annot_df['Left-Fundus'].isin(discarded_sheet['Left-Fundus']))].drop("Left-Fundus", axis=0)
# dropped = annot_df.drop(duplicates.index, axis=0)
# dropped = annot_df['Left-Fundus'].drop(discarded_sheet['Left-Fundus'])
# dropped.loc[(dropped['Left-Fundus'].isin(discarded_sheet['Left-Fundus']))]
# left_eyes = list(discarded_sheet.iloc[:, 1])
# len(left_eyes)
# annot_df.loc[(annot_df['Left-Diagnostic Keywords'].str.contains("|".join(["lens dust", "low image", "anterior", "optic disk", "refractive"])))]

In [ ]:
# drop occluded and unclear images
discarded_sheet = pd.read_excel(discarded_path, sheet_name="Sheet1")
disc_tokens = ["low image", "lens dust", "anterior", "refractive","invisible","image", "image offset","chorioretinopathy"]

annot_df_ = annot_df.loc[~annot_df['Left-Fundus'].isin(discarded_sheet['Left-Fundus'])]
annot_df = annot_df.loc[~annot_df['Right-Fundus'].isin(discarded_sheet['Right-Fundus'])]
annot_df = annot_df.loc[(~annot_df['Left-Diagnostic Keywords'].str.
                         contains("|".join(disc_tokens)))]
annot_df = annot_df.loc[(~annot_df['Right-Diagnostic Keywords'].str.
                         contains("|".join(disc_tokens)))]
# test if such exist
print(len(annot_df.loc[(annot_df['Left-Diagnostic Keywords'].str.
                        contains("|".join(disc_tokens)))]))
len(annot_df)

In [ ]:
total_count, labels_count = count_labels(annot_df, True)
print('-'*10)
occ_total_count, occ_labels_count = count_labels(occ_df, False)
total_count += occ_total_count
labels_count = [(l1+l2) for l1, l2 in zip(labels_count, occ_labels_count)]
print(labels_count)

### Retinal Image Distribution per Class

In [ ]:
init_notebook_mode(connected=False)
configure_plotly_browser_state()

labels_ser = pd.Series(total_count, name='label')
labels_df = pd.DataFrame({
    'labels': labels_ser.value_counts().index,
    "count": labels_ser.value_counts()
})

label_count = sorted(labels_count, reverse=True)
labels_df['percentage'] = [float(i / sum(label_count)) for i in label_count]
print(labels_df['percentage'])

fig = plt.figure(figsize=(30, 14))
fig = go.Figure(data=[go.Bar(
                x=labels_df['labels'],
                y=labels_df['count'],
                text=labels_df['percentage'].apply(lambda x: "{0:1.2f}%".format(x * 100)),
                textposition='auto',
                marker=dict(
                    colorscale='Viridis'
                )                
              )])

fig.update_layout(
    title='Fundus Image Distribution',
    xaxis_title='diseases',
    yaxis_title='count',
    font=dict(
        size=13
    )
)

fig.show()

In [ ]:
sns.countplot(total_count)

#### Load and Split Data

In [ ]:
# drop non-existent images
non = []
occ_non = []
for location in tqdm(annot_df.iloc[:]['Left-Fundus']):
    if not os.path.exists(os.path.join(prep_train, location)):
        non.append(location)
        
for location in tqdm(annot_df.iloc[:]['Right-Fundus']):
    if not os.path.exists(os.path.join(prep_train, location)):
        non.append(location)
        
for location in tqdm(occ_df.iloc[:]['filename']):
    if not os.path.exists(os.path.join(occ_path, location)):
        occ_non.append(location)
    
annot_df = annot_df.loc[(~annot_df['Left-Fundus'].str.match('|'.join(non)))]
annot_df = annot_df.loc[(~annot_df['Right-Fundus'].str.match('|'.join(non)))]
# occ_df = occ_df.loc[(~occ_df['filename'].str.match('|'.join(occ_non)))]

print('odir_df: ', len(annot_df))
# print('occ_df: ', len(occ_df))

**CLAHE Enhancement**

In [ ]:
def crop_image(image, plot=False, image_size=(256, 256)):
  # mask of colored pixels
  mask = image > 0

  # coordinates of colored pixels
  coordinates = np.argwhere(mask)

  # binding box of non-black pixels
  x0, y0, s0 = coordinates.min(axis=0)
  x1, y1, s1 = coordinates.max(axis=0) + 1 # slices are exclusive at the top

  # get the contents of the bounding box
  cropped = image[x0:x1, y0:y1]

  # convert to YUV for equalization
  img_yuv = cv2.cvtColor(cropped, cv2.COLOR_RGB2YUV)

  # apply adaptive equalization
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
  img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])

  # apply image normalization
  mask = np.zeros((256, 256))
  normalized = cv2.normalize(img_yuv[:,:,0], mask, 0, 255, cv2.NORM_MINMAX)

  # convert back to rgb
  new_image = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

  new_image = cv2.resize(new_image, image_size, interpolation=cv2.INTER_AREA)
  cropped_resized = cv2.resize(cropped, image_size, interpolation=cv2.INTER_AREA)
  if plot:
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title(f"Original - {image.shape}")
    plt.axis('off')
    plt.grid(False)

    # ----------------- #

    plt.subplot(1,2,2)
    plt.imshow(new_image)
    plt.title(f"Preprocessed - {new_image.shape}")
    plt.axis('off')
    plt.grid(False)
    plt.show()
  
  return cropped_resized

In [ ]:
import traceback
test_direc = os.path.join(test_folder, 'myopia')
test_files = os.listdir(os.path.join(test_folder, "myopia"))
try:
  rand_idx = np.random.randint(0, len(glaucoma_data))
  image = cv2.imread(os.path.join(glaucoma_ds, glaucoma_data[rand_idx]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  new_image = crop_image(image, plot=True, image_size=(256, 256))
except Exception as exc:
  traceback.print_tb(exc.__traceback__)

##### Preprocess Glaucoma Dataset

In [ ]:
glaucoma_prepped = []

try:
    for img in tqdm(os.listdir(glaucoma_ds)):
        img_path = os.path.join(glaucoma_ds, img)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            new_image = crop_image(image, plot=False, image_size=(IMG_SIZE, IMG_SIZE))
            glaucoma_prepped.append(new_image)
except Exception as exc:
    traceback.print_tb(exc.__traceback__)
    
print("length: ", len(glaucoma_prepped))
plt.rcParams['figure.figsize'] = (6.0, 6.0)
rand_idx = np.random.randint(0, len(glaucoma_prepped))
plt.imshow(glaucoma_prepped[rand_idx])
plt.title("shape: {0}".format(glaucoma_prepped[rand_idx].shape))
plt.grid(False)
plt.axis("off")
plt.show()

# Step 3: Data Mapping
* Mapping retinal images to their respective classes (disease)
* Start with two classes (Cataract and Normal)

In [ ]:
# upsample function
def upsample_columns(df, max_size):
    df = df.sample(frac=1).reset_index(drop=True)
    perc = len(df) / max_size
    
    size = int(max_size - (perc * max_size))
    print("upsampled by: ", size)
    for i in range(size):
        rand_idx = np.random.randint(0, len(df))
#         eye = df[rand_idx]
        eye = df.at[rand_idx, 'Left-Fundus']
        row = df.loc[df['Left-Fundus'].str.match(eye)]
        
        if len(row) > 1:
            df = df.append(row.iloc[0], ignore_index=True)
        else:
            df = df.append(row, ignore_index=True)
#         df = pd.concat([df, row]).reset_index(drop=True)
#     df = pd.Series(df)
    return df

In [ ]:
# eye = df_hypertension.at[322, 'Left-Fundus']
# row = df_hypertension.loc[df_hypertension['Left-Fundus'].str.match(eye)]
# # conc = pd.concat([df_hypertension, row])
# conc = df_hypertension.append(row.iloc[0], ignore_index=True)
# # row.iloc[0]
# conc

In [ ]:
# print(df_slim.at[432, 'Left-Fundus'])

In [ ]:
# slim to necessary columns
df_slim = annot_df.iloc[:, 1:15]
df_slim.head()

### Extract Cataract Images (Left and Right Retinas)

In [ ]:
# extract left cataract images
df_left_cat = df_slim[df_slim['Left-Diagnostic Keywords'].str.match('cataract')]
df_left_cat.head()

In [ ]:
# extract right cataract images
df_right_cat = df_slim[df_slim['Right-Diagnostic Keywords'].str.match('cataract')]
df_right_cat.head()

In [ ]:
annot_df.loc[annot_df['D'] == 1, 'Left-Fundus']

### Combine Cataract Left and Right Images

In [ ]:
# df_cat_filenames = df_left_cat['Left-Fundus'].append(df_right_cat['Right-Fundus'], ignore_index=True)
df_cat_filenames = pd.concat([df_left_cat, df_right_cat])
df_cat_filenames.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
# df_cat_filenames = df_cat_filenames.sample(n=485).reset_index(drop=True)
df_cat_filenames.head()

In [ ]:
df_cat_filenames.tail()

In [ ]:
len(df_cat_filenames)

### Extract Normal Retinal Images (Left and Right)

In [ ]:
# extract left norms
df_left_norm = df_slim.loc[df_slim['Left-Diagnostic Keywords'].str.match('normal')]
df_left_norm.head()

In [ ]:
# extract right norm
df_right_norm = df_slim.loc[df_slim['Right-Diagnostic Keywords'].str.match('normal')]
df_right_norm.head()

### Combine Normal Images (Left and Right)

In [ ]:
# df_norm_filenames = df_left_norm['Left-Fundus'].append(df_right_norm['Right-Fundus'], ignore_index=True)
df_norm_filenames = pd.concat([df_left_norm, df_right_norm])
df_norm_filenames.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_norm_filenames.head()

In [ ]:
df_norm_filenames.tail()

In [ ]:
len(df_norm_filenames)

In [ ]:
# extract diabetes
df_left_diabetes = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('|'.join(['diabetic', 'proliferative'])))]
df_right_diabetes = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('|'.join(['diabetic', 'proliferative'])))]

# combine diabetes
# df_diabetes = df_left_diabetes['Left-Fundus'].append(df_right_diabetes['Right-Fundus'], ignore_index=True)
df_diabetes = pd.concat([df_left_diabetes, df_right_diabetes])
df_diabetes.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
print("diabetes: ", len(df_diabetes))

In [ ]:
# extract glaucoma
df_left_glaucoma = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('glaucoma'))]
df_right_glaucoma = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('glaucoma'))]

# combine glaucoma
# df_glaucoma = df_left_glaucoma.append(df_right_glaucoma, ignore_index=True)
df_glaucoma = pd.concat([df_left_glaucoma, df_right_glaucoma])
df_glaucoma.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_glaucoma = upsample_columns(df_glaucoma, len(df_cat_filenames))
print("glaucoma: ", len(df_glaucoma))

In [ ]:
# extract amd
df_left_amd = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('age-related'))]
df_right_amd = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('age-related'))]

# combine amd
# df_amd = df_left_amd['Left-Fundus'].append(df_right_amd['Right-Fundus'], ignore_index=True)
df_amd = pd.concat([df_left_amd, df_right_amd])
df_amd.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'],axis='columns', inplace=True)
df_amd = upsample_columns(df_amd, len(df_cat_filenames))
print("amd: ", len(df_amd))

In [ ]:
# extract hypertension
df_left_hypertension = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('hypertensive'))]
df_right_hypertension = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('hypertensive'))]

# combine hypertension 
# df_hypertension = df_left_hypertension['Left-Fundus'].append(df_right_hypertension['Right-Fundus'], ignore_index=True)
df_hypertension = pd.concat([df_left_hypertension,df_right_hypertension]).reset_index(drop=True)
df_hypertension.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_hypertension = upsample_columns(df_hypertension, len(df_cat_filenames))
print("hypertension: ", len(df_hypertension))

In [ ]:
# extract myopia
df_left_myopia = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('pathological myopia'))]
df_right_myopia = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('pathological myopia'))]

# combine myopia
# df_myopia = df_left_myopia['Left-Fundus'].append(df_right_myopia['Right-Fundus'], ignore_index=True)
df_myopia = pd.concat([df_left_myopia, df_right_myopia])
df_myopia.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_myopia = upsample_columns(df_myopia,len(df_cat_filenames))
# df_myopia = upsample_columns(df_myopia,970)
print("myopia: ", len(df_myopia))

In [ ]:
common_diseases = ['myopia', 'normal fundus', 'cataract', 'glaucoma', \
                   'hypertensive', 'proliferative', 'age-related', 'diabetic']
df_left_others = df_slim.loc[(~df_slim['Left-Diagnostic Keywords'].str.contains('|'.join(common_diseases)))]
df_right_others = df_slim.loc[(~df_slim['Right-Diagnostic Keywords'].str.contains('|'.join(common_diseases)))]

# combine others
# df_others = df_left_others['Left-Fundus'].append(df_right_others['Right-Fundus'], ignore_index=True)
df_others = pd.concat([df_left_others, df_right_others])
df_others.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_others = upsample_columns(df_others, len(df_cat_filenames))
print("others: ", len(df_others))

### Select and Create a Random Sample
* we need to minimize the normal data to match the number of cataract samples

In [ ]:
# There are 572 cataract images
df_norm_filenames_random = df_norm_filenames.sample(n=485)
df_norm_filenames_random.head()

In [ ]:
df_norm_filenames_random = df_norm_filenames_random.reset_index(drop=True)
df_norm_filenames_random

In [ ]:
##### df_normal_dn = df_normal.sample(n=381).reset_index(drop=True)
df_diabetes_dn = df_diabetes.sample(n=485).reset_index(drop=True)
df_others_dn = df_others.sample(n=485).reset_index(drop=True)
df_myopia_dn = df_myopia.sample(n=485).reset_index(drop=True)
df_amd_dn = df_amd.sample(n=485).reset_index(drop=True)
df_glaucoma_dn = df_glaucoma.sample(n=485).reset_index(drop=True)
# df_cataract_dn = df_cataract.sample(n=381).reset_index(drop=True)


In [ ]:
# # df_normal_dn = df_normal.sample(n=381).reset_index(drop=True)
# df_diabetes_dn = df_diabetes.sample(n=458).reset_index(drop=True)
# df_others_dn = df_others.sample(n=489).reset_index(drop=True)
# df_myopia_dn = df_myopia.sample(n=478).reset_index(drop=True)
# df_amd_dn = df_amd.sample(n=469).reset_index(drop=True)
# df_glaucoma_dn = df_glaucoma.sample(n=494).reset_index(drop=True)
# # df_cataract_dn = df_cataract.sample(n=381).reset_index(drop=True)


In [ ]:
df_glaucoma_dn.iloc[1, 0] # [row, col]

In [ ]:
np.random.randint(0, 2)

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0.1, hspace=0.2)

for i in range(9):
    rand_idx = np.random.randint(0, len(df_myopia_dn))
    img_file = df_myopia_dn.iloc[rand_idx, np.random.randint(0, 2)]
    img = cv2.imread(os.path.join(prep_train, img_file))
    plt.subplot(3,3,i+1)
    plt.title(f"{img_file.split('.')[0]}: myopia")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.grid(False)
    
plt.show()

In [ ]:
dframe_fundus = pd.concat([df_norm_filenames_random, df_cat_filenames, df_glaucoma_dn, df_myopia_dn])
dframe_fundus.iloc[:, 2:10].sum()

In [ ]:
# count labels
def extract_labels(df, initial):
    labels = []
    fundus_names = df.iloc[:]
    
    for i in range(len(fundus_names)):
        labels.append(initial)
        
    return labels

## Visualize Class Distribution (After data mapping)

In [ ]:
total_count_labels = []
labels_each_count = []

labels_normal = extract_labels(df_norm_filenames_random, "N")
labels_diabetes = extract_labels(df_diabetes_dn, "D")
labels_glaucoma = extract_labels(df_glaucoma_dn, "G")
labels_cataract = extract_labels(df_cat_filenames, "C")
labels_amd = extract_labels(df_amd_dn, "A")
labels_hypertension = extract_labels(df_hypertension, "H")
labels_myopia = extract_labels(df_myopia_dn, "M")
labels_others = extract_labels(df_others_dn, "O")

total_count_labels += labels_normal
# total_count_labels += labels_diabetes
total_count_labels += labels_glaucoma
total_count_labels += labels_cataract
# total_count_labels += labels_amd
# total_count_labels += labels_hypertension
total_count_labels += labels_myopia
# total_count_labels += labels_others

labels_each_count.append(len(labels_normal))
# labels_each_count.append(len(labels_diabetes))
labels_each_count.append(len(labels_glaucoma))
labels_each_count.append(len(labels_cataract))
# labels_each_count.append(len(labels_amd))
# labels_each_count.append(len(labels_hypertension))
labels_each_count.append(len(labels_myopia))
# labels_each_count.append(len(labels_others))

print("Total Count: ", len(total_count_labels))
labels_each_count

In [ ]:
total_count_labels += occ_total_count
labels_each_count = [(l1+l2) for l1, l2 in zip(labels_each_count, occ_labels_count)]
print(labels_each_count)

In [ ]:
init_notebook_mode(connected=False)
configure_plotly_browser_state()

labels_ser = pd.Series(total_count_labels, name='label')
labels_df = pd.DataFrame({
    'labels': labels_ser.value_counts().index,
    'count': labels_ser.value_counts()
})

labels_count = sorted(labels_each_count, reverse=True)
labels_df['percentage'] = [float(i / sum(labels_count)) for i in labels_count]

fig = plt.figure(figsize=(16, 16))
fig = go.Figure(data=[go.Bar(
                x=labels_df['labels'],
                y=labels_df['count'],
                text=labels_df['percentage'].apply(lambda x: "{0:1.2f}%".format(x * 100)),
                textposition='auto',
                marker=dict(
                    colorscale='Viridis'
                )
                )])

fig.update_layout(
    title='Resampled Fundus Image Distribution',
    xaxis_title='disease',
    yaxis_title='count',
    font=dict(
        size=13
    )
)

fig.show()

#### Split Data (After resampling)

In [ ]:
# check for non-existent

# not_exists = []

# for file in tqdm(dframe_fundus.iloc[:]['Left-Fundus']):
#   if not os.path.exists(os.path.join(prep_train, file)):
#     not_exists.append(file)

# for file in tqdm(dframe_fundus.iloc[:]['Right-Fundus']):
#   if not os.path.exists(os.path.join(prep_train, file)):
#     not_exists.append(file)

# dframe_fundus = dframe_fundus.loc[(~dframe_fundus['Left-Fundus'].str.match('|'.join(not_exists)))]
# dframe_fundus = dframe_fundus.loc[(~dframe_fundus['Right-Fundus'].str.match('|'.join(not_exists)))]

# len(dframe_fundus)

In [ ]:
dframe_fundus.head()

In [ ]:
dframe_fundus.iloc[:, 2:10].sum()

In [ ]:
left = []
right = []
occ_files = []

# dframe_fundus = pd.concat([df_norm_filenames_random, df_cat_filenames, df_diabetes_dn,
#                           df_glaucoma_dn, df_myopia_dn, df_hypertension, df_amd_dn,
#                           df_others_dn]).reset_index(drop=True)

dframe_fundus = pd.concat([df_norm_filenames_random, df_cat_filenames, df_glaucoma_dn, df_myopia_dn])

dframe_fundus = dframe_fundus.sample(frac=1).reset_index(drop=True)
occ_dframe = occ_df.sample(frac=1).reset_index(drop=True)

# dframe_fundus = dframe_fundus.sample(n=2600).reset_index(drop=True)
# dframe_fundus.tail(5)

# check if all filenames exist


for file in tqdm(dframe_fundus.iloc[:]['Left-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
#         img = cv2.imread(os.path.join(prep_train, file), cv2.IMREAD_UNCHANGED)
        img = cv2.imread(os.path.join(prep_train, file), 0)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 1)
        left.append(img)
        
for file in tqdm(dframe_fundus.iloc[:]['Right-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
        img = cv2.imread(os.path.join(prep_train, file), 0)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 1)
        right.append(img)
        
for file in tqdm(occ_dframe.iloc[:]['filename']):
    if os.path.isfile(os.path.join(occ_path, file)):
        img = cv2.imread(os.path.join(occ_path, file), 0)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 1)
        occ_files.append(img)

X0 = np.array(occ_files)
X1 = np.array(left)
X2 = np.array(right)

In [ ]:
print(X0.shape, X1.shape)

In [ ]:
# RGB

left1 = []
right1 = []
occ_files1 = []
for file in tqdm(dframe_fundus.iloc[:]['Left-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
        img = cv2.imread(os.path.join(prep_train, file), cv2.IMREAD_UNCHANGED)
#         img = cv2.imread(os.path.join(prep_train, file), 0)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
        left1.append(img)
        
for file in tqdm(dframe_fundus.iloc[:]['Right-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
        img = cv2.imread(os.path.join(prep_train, file), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
        right1.append(img)
        
for file in tqdm(occ_dframe.iloc[:]['filename']):
    if os.path.isfile(os.path.join(occ_path, file)):
        img = cv2.imread(os.path.join(occ_path, file), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
        occ_files1.append(img)
    
X3 = np.array(left1)
X4 = np.array(right1)
X5 = np.array(occ_files1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

classes = list(dframe_fundus.iloc[:, 2:11].sum().index)
# y = np.array(dframe_fundus.iloc[:][classes])
y = np.array(dframe_fundus.iloc[:][['N', 'C', 'G', 'M']])
y = np.array(y)
y1 = np.array(occ_dframe.iloc[:][['N', 'C', 'G', 'M']])
y = np.concatenate((y, y, y1))
print("Labels: " + str(y.shape))

**Train Test Splitting**

**Singular Inputs**

In [ ]:
from sklearn.model_selection import train_test_split

X = np.concatenate((X1, X2, X0))
# X_norm = X / 255

x_train, x_test, \
y_train, y_test = train_test_split(X, y, test_size=0.20)

x_train, x_valid, \
y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10)

X_RGB = np.concatenate((X3, X4, X5))
# X_norm = X / 255

x_train1, x_test1, \
y_train1, y_test1 = train_test_split(X_RGB, y, test_size=0.20)

x_train1, x_valid1, \
y_train1, y_valid1 = train_test_split(x_train1, y_train1, test_size=0.10)


# Step 5: Build Network Architecture (Toy ResNet)

#### BatchNorm CReLU ResNet

In [ ]:
from tensorflow.keras import layers

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='img')
x = layers.Conv2D(32, 3, dtype='float32')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
x = layers.Conv2D(64, 3)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, padding='same')(block_1_output)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_2_output = layers.add([x, block_1_output])
activation_1 = layers.Activation(tf.nn.relu)(block_2_output)

x = layers.Conv2D(64, 3, padding='same')(activation_1)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_3_output = layers.add([x, activation_1])
activation_2 = layers.Activation(tf.nn.relu)(block_3_output)

x = layers.Conv2D(64, 3, padding='same')(activation_2)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_4_output = layers.add([x, activation_2])
activation_3 = layers.Activation(tf.nn.relu)(block_4_output)

x = layers.Conv2D(64, 3, padding='same')(activation_3)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
x = layers.Conv2D(96, 3)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
x = layers.Conv2D(128, 3)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
block_5_output = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.Conv2D(128, 3, padding='same')(block_5_output)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_6_output = layers.add([x, block_5_output])
activation_4 = layers.Activation(tf.nn.relu)(block_6_output)

x = layers.Conv2D(128, 3, padding='same')(activation_4)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_7_output = layers.add([x, activation_4])
activation_5 = layers.Activation(tf.nn.relu)(block_7_output)

x = layers.Conv2D(128, 3, padding='same')(activation_5)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_8_output = layers.add([x, activation_5])
activation_6 = layers.Activation(tf.nn.relu)(block_8_output)

x = layers.Conv2D(128, 3, padding='same')(activation_6)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
x = layers.Conv2D(192, 3)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
x = layers.Conv2D(256, 3)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.relu)(x)
block_9_output = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.Conv2D(256, 3, padding='same')(block_9_output)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_10_output = layers.add([x, block_9_output])
activation_7 = layers.Activation(tf.nn.relu)(block_10_output)

x = layers.Conv2D(256, 3, padding='same')(activation_7)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_11_output = layers.add([x, block_10_output])
activation_8 = layers.Activation(tf.nn.relu)(block_11_output)

x = layers.Conv2D(256, 3, padding='same')(activation_8)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
block_12_output = layers.add([x, block_11_output])
activation_9 = layers.Activation(tf.nn.relu)(block_12_output)

x = layers.Conv2D(256, 3)(activation_9)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(tf.nn.crelu)(x)
x = layers.Conv2D(256, 3, activation=tf.nn.relu, name='conv2d_end')(x)
x = layers.GlobalAveragePooling2D(name='global_average_pooling2d')(x)
x = layers.Dense(256, activation=tf.nn.relu)(x)
# x = layers.Dropout(0.3)(x)
outputs = layers.Dense(4, activation=tf.nn.softmax, name='dense_output')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='resnet_batch-norm_crelu')
# model.summary()
print("Layers: ", len(model.layers))

### Plot Model Architecture

In [ ]:
# from tensorflow.keras.models import load_model

# custom_objects = {"custom_activation": tf.nn.crelu}
# # model = load_model(os.path.join(model_path, "odir_checkpoint.h5"), custom_objects=custom_objects)
# # model1 = load_model(os.path.join(model_path, "odir_model-{0:.2f}".format(test_acc)))
# model = load_model(odir_model_merged)
# # model1.load_weights(model_path, "odir_model-{0:.2f}-bw".format(test_acc))
# # model = load_model(odir_model_dir)

# # model = tf.saved_model.load(model_path)
# model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from matplotlib import image as mpimg

model_path = "trained_models/odir"
if os.path.isdir(model_path):
  clear_content(model_path)
else:
  os.makedirs(model_path)

model_diagram_path = os.path.join(model_path, "odir_model.png")

plot_model(model, model_diagram_path,
           show_shapes=True,
           show_layer_names=True)

# show the plot
img = mpimg.imread(model_diagram_path)
plt.figure(figsize=(30, 20))
plt.axis('off')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
from tensorflow.keras import backend as K

def weighted_categorical_crossentropy(weights):
  """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
      weights: numpy array of shape(C,) where C is the number of classes
      
    Usage:
      weights = np.array([0.5, 2, 10]) # Class one at 0.5, 2 is twice the normal weights,
                                        10 is 10x
      loss = weighted_categorical_crossentropy(weights)
      model.compile(loss=loss, optimizer='adam')

    Returns:
      loss: weighted loss of categorical data
    """

  weights = K.variable(weights)
  def loss(y_true, y_pred):
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

    # clip to prevents NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())

    # calc
    y_true = y_true.astype('float64')
#     print(y_true, K.log(y_pred))
#     print(y_train.dtype, y_pred.dtype)
    print(tf.math.round(K.log(y_pred)).dtype)
    
    loss = y_true * tf.math.round(K.log(y_pred)) * weights
#     loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, -1)
    print(y_true.dtype, y_pred.dtype)
    
    return loss

  return loss

In [ ]:
"""
Define our custom loss function.
"""
import numpy as np
from tensorflow.keras import backend as K
import tensorflow as tf

import dill


def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        y_true = tf.cast(y_true, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss

    return binary_focal_loss_fixed


def categorical_focal_loss(alpha, gamma=2.):
    """
    Softmax version of focal loss.
    When there is a skew between different categories/labels in your data set, you can try to apply this function as a
    loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
      categories/labels, the size of the array needs to be consistent with the number of classes.
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    alpha = np.array(alpha, dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """
        y_true, y_pred = np.argmax(y_true), np.argmax(y_pred)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed

In [ ]:
dframe_fundus.info()

In [ ]:
tf.reduce_mean(tf.square(tf.subtract(len(df_norm_filenames), len(dframe_fundus))))

In [ ]:
NUM_EPOCHS = 3
BATCH_SIZE=32

# dframe_fundus = pd.concat([df_norm_filenames_random, df_cat_filenames, df_diabetes_dn,
#                           df_glaucoma_dn, df_myopia_dn, df_hypertension, df_amd_dn,
#                           df_others_dn]).reset_index(drop=True)

N = float(len(df_norm_filenames_random)/len(dframe_fundus))
C = float(len(df_cat_filenames)/len(dframe_fundus))
G = float(len(df_glaucoma_dn)/len(dframe_fundus))
D = float(len(df_diabetes_dn)/len(dframe_fundus))
M = float(len(df_myopia_dn)/len(dframe_fundus))
H = float(len(df_hypertension)/len(dframe_fundus))
A = float(len(df_amd_dn)/len(dframe_fundus))
O = float(len(df_others_dn)/len(dframe_fundus))

# weights = tf.constant([0.2, 0.2, 0.2, 0.2])
# weights = tf.constant([N, D, G, C, A, H, M, O])
weights = tf.constant([N, C, G,  M])
weights = tf.cast(weights, tf.float32)

weighted_loss = weighted_categorical_crossentropy(weights)

loss_fn = keras.losses.CategoricalCrossentropy()
# focal_loss_categorical = categorical_focal_loss(gamma=2, alpha=[[.15,.15,.125,.125,.15,.15,.075, .075]])
focal_loss_categorical = categorical_focal_loss(gamma=2, alpha=[[.30, .20, .20, .15, .15]])
metrics = [
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.AUC(name='auc')
]


rms_prop = keras.optimizers.RMSprop(lr=LR, decay=LR/NUM_EPOCHS)
sgd_momentum = keras.optimizers.SGD(lr=0.01, decay=0.01/NUM_EPOCHS, \
                                    momentum=0.9, nesterov=True)
adadelta = keras.optimizers.Adadelta()

model.compile(loss=weighted_loss,
              optimizer=rms_prop,
              metrics=metrics)

# Step 6: Train the Model

In [ ]:
# callbacks
import traceback
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

checkpoint_dir = os.path.join(model_path, "odir_checkpoint.ckpt")
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                             monitor='val_accuracy',
                                             mode='max',
                                             save_best_only=True,
                                             verbose=1)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               mode='auto',
                                               min_delta=0,
                                               patience=100,
                                               restore_best_weights=True,
                                               verbose=1)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                              mode='min',
                                              min_delta=0.0001,
                                              factor=0.1,
                                              patience=140,
                                              verbose=1)

# model_hist = model.fit_generator(train_generator,
#                                  epochs=NUM_EPOCHS,
#                                  steps_per_epoch=(train_generator.samples // BATCH_SIZE),
#                                  callbacks=[
#                                             checkpoint,
#                                             early_stopping
#                                  ],
#                                  validation_data=(valid_generator),
#                                  validation_steps=(valid_generator.samples // BATCH_SIZE))
# try:
# model_hist = model.fit([x_train1, x_train2], y_train, 
#                        epochs=NUM_EPOCHS, 
#                        batch_size=BATCH_SIZE,
#                        steps_per_epoch=((len(x_train1) + len(x_train2)) // BATCH_SIZE),
#                        callbacks=[
#                            checkpoint,
#                            early_stopping
#                        ],
#                        verbose=1, shuffle=True, 
#                        validation_data = ([x_valid1, x_valid2], y_valid),
#                        validation_steps=((len(x_valid1) + len(x_valid2)) // BATCH_SIZE))


# try:
model_hist = model.fit(x_train1, y_train1,
                       epochs=NUM_EPOCHS,
                       batch_size=BATCH_SIZE,
                       steps_per_epoch=((len(x_train1) // BATCH_SIZE)),
                       callbacks=[
                                  checkpoint,
                                  early_stopping
                       ],
                       verbose=1,
                       shuffle=True,
                       validation_data=(x_valid1, y_valid1),
                       validation_steps=(len(x_valid1) // BATCH_SIZE)
                       )
# except Exception as exc:
#     traceback.print_tb(exc.__traceback__)

### Save Model

In [ ]:
_, train_acc, train_prec, train_auc = model.evaluate(x_train1, y_train1)
_, test_acc, test_prec, test_auc = model.evaluate(x_test1, y_test1)

print("Train Accuracy: {0: .2f}% - Train Precision: {1: .2f}% - Train AUC: {2:.2f}%".format(train_acc * 100, train_prec * 100, train_auc * 100))
print("Test Accuracy: {0:.2f}% - Test Precision: {1: .2f}% - Test AUC: {2: .2f}%".format(test_acc * 100, test_prec * 100, test_auc * 100))

**For Custom Objects in Model**

In [ ]:
# def crelu_fn(x):
#   x = np.concatenate((x, -x), axis=1)
#   return tf.nn.relu(x)

# config = model.get_config()

# custom_objects = {"custom_activation": tf.nn.crelu}
# with keras.utils.custom_object_scope(custom_objects):
#   model = keras.Model.from_config(config)

In [ ]:
# import pickle

# model.save(os.path.join(model_path, "odir_model-{0:.2f}".format(test_acc)))
# model.save_weights(os.path.join(model_path, "odir_model-{0:.2f}-bw".format(test_acc)))

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()
# open(os.path.join(model_path, "odir_model_four_{0:.2f}.tflite".format(test_acc)), 'wb').write(tflite_model)
# print("Model converted to TFLite Model!")

# tf.saved_model.save(model, model_path)
# print("Saved model as SavedModel Format!")

# pickle_out = open(os.path.join(model_path, "odir_history.h5"), 'wb')
# pickle.dump(saved_history, pickle_out)
# pickle_out.close()

In [ ]:
for dirname, subfolders, filenames in os.walk(model_path):
    for file in filenames:
        print(os.path.join(dirname, file))

In [ ]:
# Load saved pickle
import pickle

# pickle_in = open(os.path.join(model_path, "odir_history.h5"), "rb")
pickle_in = open(odir_history_merged, "rb")
saved_history = pickle.load(pickle_in)
len(saved_history)

In [ ]:
import pickle

model.save(os.path.join(model_path, "odir_model-{0:.2f}".format(test_acc)))
model.save_weights(os.path.join(model_path, "odir_model-{0:.2f}-bw".format(test_acc)))

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(os.path.join(model_path, "odir_model_four_{0:.2f}.tflite".format(test_acc)), 'wb').write(tflite_model)
print("Model converted to TFLite Model!")

tf.saved_model.save(model, model_path)
print("Saved model as SavedModel Format!")

pickle_out = open(os.path.join(model_path, "odir_history.h5"), 'wb')
pickle.dump(saved_history, pickle_out)
pickle_out.close()

In [ ]:
# saved_history = model_hist.history

### Download Model

In [ ]:
model_path

In [ ]:
from zipfile import ZipFile
from IPython.display import FileLink
model_obj = ZipFile("odir_rgb_merged_model_four-{0:.4f}.zip".format(test_acc), "w")

# for file in os.listdir(model_path):
#   model_obj.write(os.path.join(model_path, file))

for dirname, subfolders, filenames in os.walk(model_path):
    for file in filenames:
        model_obj.write(os.path.join(dirname, file))

model_obj.close()
FileLink("odir_rgb_merged_model_four-{0:.4f}.zip".format(test_acc))
# shutil.move("odir_model_eight-{0:.4f}.zip".format(test_acc), "/content/drive/MyDrive/Colab Notebooks/thesis/odir_model_eight-{0:.4f}.zip".format(test_acc))

# Step 7: Performance Analysis

In [ ]:
from tensorflow.keras.models import load_model

custom_objects = {"custom_activation": tf.nn.crelu}
# model = load_model(os.path.join(model_path, "odir_checkpoint.h5"), custom_objects=custom_objects)
# model1 = load_model(os.path.join(model_path, "odir_model-{0:.2f}".format(test_acc)))
model = load_model(odir_model_merged)
# model1.load_weights(model_path, "odir_model-{0:.2f}-bw".format(test_acc))
# model = load_model(odir_model_dir)

# # model = tf.saved_model.load(model_path)
model.summary()

In [ ]:
import plotly.graph_objs as go
# define scatter plot visualization on plotly express

NUM_EPOCHS = 150
def display_training_curves(training, validation, yaxis):
    if yaxis == "loss":
        ylabel = "Loss"
        title = "Model Loss with respect to Epochs"
    elif yaxis == "accuracy":
        ylabel = "Accuracy"
        title = "Model Accuracy with respect to Epochs"
    elif yaxis == "precision":
        ylabel = "Precision"
        title = "Model Precision with respect to Epochs"
    elif yaxis == "auc":
        ylabel = "AUC"
        title = "Model AUC with respect to Epochs"
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(x = np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers',
                  y=training, marker=dict(color='dodgerblue'), name='Training')
    )
    
    fig.add_trace(
        go.Scatter(x=np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers', y=validation, marker=dict(color='darkorange'),
                  name='Validation')
    )
    
    fig.update_layout(title=title, yaxis_title=ylabel,
                     xaxis_title='Epochs', template='plotly_white')
    fig.show()

In [ ]:
plt.style.use('ggplot')

fig1, ax = plt.subplots(4, 1)
plt.subplots_adjust(wspace=0, hspace=0.2)

line1 = ax[0].plot(saved_history['accuracy'], label='Train Accuracy')
line2 = ax[0].plot(saved_history['val_accuracy'], label="Test Accuracy")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
ax[0].set_title("Model Accuracy")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

line1 = ax[1].plot(saved_history['loss'], label='Train Loss')
line2 = ax[1].plot(saved_history['val_loss'], label='Test Loss')
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
ax[1].set_title("Model Loss")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")

line1 = ax[2].plot(saved_history['precision'], label='Train Precision')
line2 = ax[2].plot(saved_history['val_precision'], label='Test Precision')
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
ax[2].set_title("Model Precision")
ax[2].set_xlabel("Epochs")
ax[2].set_ylabel("Precision")

line1 = ax[3].plot(saved_history['auc'], label='Train AUC')
line2 = ax[3].plot(saved_history['val_auc'], label='Test AUC')
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
ax[3].set_title("Model AUC")
ax[3].set_xlabel("Epochs")
ax[3].set_ylabel("AUC")

for i in range(4):
  ax[i].legend()

fig1.canvas.set_window_title("Train vs Test")
fig1.set_figwidth(12)
fig1.set_figheight(8)

plt.tight_layout()
plt.show()

**Model Accuracy**

In [ ]:
plt.rcParams['font.size'] = 10
plt.figure(figsize=(10, 4))
line1 = plt.plot(saved_history['accuracy'], label='Train Accuracy')
line2 = plt.plot(saved_history['val_accuracy'], label="Test Accuracy")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
display_training_curves(
    saved_history['accuracy'],
    saved_history['val_accuracy'],
    'accuracy'
)

**Model Loss**

In [ ]:
plt.rcParams['font.size'] = 10
plt.figure(figsize=(10, 4))
line1 = plt.plot(saved_history['loss'], label='Train Loss')
line2 = plt.plot(saved_history['val_loss'], label="Test Loss")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
display_training_curves(
    saved_history['loss'],
    saved_history['val_loss'],
    'loss'
)

**Model Precision**

In [ ]:
plt.rcParams['font.size'] = 10
plt.figure(figsize=(10, 4))
line1 = plt.plot(saved_history['precision'], label='Train Precision')
line2 = plt.plot(saved_history['val_precision'], label="Test Preicison")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.title("Model Precision")
plt.xlabel("Epochs")
plt.ylabel("Precision")

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
display_training_curves(
    saved_history['precision'],
    saved_history['val_precision'],
    'precision'
)

**Model AUC**

In [ ]:
plt.rcParams['font.size'] = 10
plt.figure(figsize=(10, 4))
line1 = plt.plot(saved_history['auc'], label='Train AUC')
line2 = plt.plot(saved_history['val_auc'], label="Test AUC")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.title("Model AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
display_training_curves(
    saved_history['auc'],
    saved_history['val_auc'],
    'auc'
)

## Step 7.1: Confusion Matrix and Classification Report

In [ ]:
# class_labels = list(train_generator.class_indices.keys())
# print(class_labels)

In [ ]:
# class_labels = list(annot_df.iloc[:, -12:-4].sum().index)
class_labels = ['N', 'C', 'G', 'M']
# class_labels = classes

In [ ]:
# y_pred_raw = model.predict_generator(test_generator, verbose=1)
y_pred_raw = model.predict(x_test1, verbose=1)
y_pred = np.argmax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

# print("Confusion Matrix")
# print(confusion_matrix(test_generator.classes, y_pred))
print("Classfication Report")
print(classification_report(y_test_classes, y_pred, target_names=class_labels))

In [ ]:
# y_pred_raw = model.predict_generator(test_generator, verbose=1)
# y_pred_raw = model.predict(x_test, verbose=1)
# y_pred = np.argmax(y_pred_raw, axis=1)

# nb_test_samples = test_generator.samples
# nb_train_samples = train_generator.samples

y_test_classes = np.argmax(y_test1, axis=1)

plt.figure(figsize=(14, 14))
# cm = confusion_matrix(test_generator.classes, y_pred)
conf_mtrx = confusion_matrix(y_test_classes, y_pred)
cm = np.around(conf_mtrx.astype('float') / conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)

# use white text if squares are dark; otherwise black
threshold = cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(cm.shape[1]), range(cm.shape[0])):
  color = 'black' if cm[i, j] > threshold else "white"
  plt.text(j, i, cm[i, j], horizontalalignment='center', color=color)

plt.imshow(cm)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

In [ ]:
print(conf_mtrx)

**Compute Kappa**

In [ ]:
agreed = (conf_mtrx[0][0] + conf_mtrx[1][1] + conf_mtrx[2][2] + conf_mtrx[3][3]) / len(y_test)
prob_true_N = float((conf_mtrx[0][0] + conf_mtrx[1][0] + conf_mtrx[2][0] + conf_mtrx[3][0]) / len(y_test))
prob_pred_N = float((conf_mtrx[0][0] + conf_mtrx[0][1] + conf_mtrx[0][2] + conf_mtrx[0][3]) / len(y_test))
chance_N = float(prob_true_N * prob_pred_N)

prob_true_C = float((conf_mtrx[0][1] + conf_mtrx[1][1] + conf_mtrx[2][1] + conf_mtrx[3][1]) / len(y_test))
prob_pred_C = float((conf_mtrx[1][0] + conf_mtrx[1][1] + conf_mtrx[1][2] + conf_mtrx[1][3]) / len(y_test))
chance_C = float(prob_true_C * prob_pred_C)

prob_true_G = float((conf_mtrx[0][2] + conf_mtrx[1][2] + conf_mtrx[2][2] + conf_mtrx[3][2]) / len(y_test))
prob_pred_G = float((conf_mtrx[2][0] + conf_mtrx[2][1] + conf_mtrx[2][2] + conf_mtrx[2][3]) / len(y_test))
chance_G = float(prob_true_G * prob_pred_G)

prob_true_M = float((conf_mtrx[0][3] + conf_mtrx[1][3] + conf_mtrx[2][3] + conf_mtrx[3][3]) / len(y_test))
prob_pred_M = float((conf_mtrx[3][0] + conf_mtrx[3][1] + conf_mtrx[3][2] + conf_mtrx[3][3]) / len(y_test))
chance_M = float(prob_true_M * prob_pred_M)

chance_agreed = float(chance_N + chance_C + chance_G + chance_M)

kappa_score = float((agreed - chance_agreed)/(1-chance_agreed))
print("Kappa Score: {0:.4f}".format(kappa_score))

In [ ]:
agreed

**Class Evaluation**

In [ ]:

# Normal Evaluation
N_tp = conf_mtrx[0][0]
N_fn = (conf_mtrx[1][0] + conf_mtrx[2][0] + conf_mtrx[3][0])
N_tn = (conf_mtrx[1][1] + conf_mtrx[1][2] + conf_mtrx[1][3] + \
        conf_mtrx[2][1] + conf_mtrx[2][2] + conf_mtrx[2][3] + \
        conf_mtrx[3][1] + conf_mtrx[3][2] + conf_mtrx[3][3])
N_fp = (conf_mtrx[0][1] + conf_mtrx[0][2] + conf_mtrx[0][3])

N_tpr = N_tp / (N_tp + N_fn)
N_tnr = N_tn / (N_tn + N_fp)
N_fpr = N_fp / (N_fp + N_tn)
N_fnr = N_fn / (N_fn + N_tp)

print("Normal Category: ")
print("-->  True Positives: ", N_tp)
print("-->  True Negatives: ", N_tn)
print("-->  False Positives: ", N_fp)
print("-->  False Negatives: ", N_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(N_tpr))
print("-->  True Negative Rate: {0:.4f}".format(N_tnr))
print("-->  False Positive Rate: {0:.4f}".format(N_fpr))
print("-->  False Negative Rate: {0:.4f}".format(N_fnr))

print()
print('=*=*'*8)
print()

# Cataract Evaluation
C_tp = conf_mtrx[1][1]
C_tn = (conf_mtrx[0][0] + conf_mtrx[0][2] + conf_mtrx[0][3] + \
        conf_mtrx[2][0] + conf_mtrx[2][2] + conf_mtrx[2][3] + \
        conf_mtrx[3][0] + conf_mtrx[3][2] + conf_mtrx[3][3])
C_fp = (conf_mtrx[1][0] + conf_mtrx[1][2] + conf_mtrx[1][3])
C_fn = (conf_mtrx[0][1] + conf_mtrx[2][1] + conf_mtrx[3][1])

C_tpr = C_tp / (C_tp + C_fn)
C_tnr = C_tn / (C_tn + C_fp)
C_fpr = C_fp / (C_fp + C_tn)
C_fnr = C_fn / (C_fn + C_tp)

print("Cataract Category: ")
print("-->  True Positives: ", C_tp)
print("-->  True Negatives: ", C_tn)
print("-->  False Positives: ", C_fp)
print("-->  False Negatives: ", C_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(C_tpr))
print("-->  True Negative Rate: {0:.4f}".format(C_tnr))
print("-->  False Positive Rate: {0:.4f}".format(C_fpr))
print("-->  False Negative Rate: {0:.4f}".format(C_fnr))

print()
print('=*=*'*8)
print()

# Glaucoma Evaluation
G_tp = conf_mtrx[2][2]
G_tn = (conf_mtrx[0][0] + conf_mtrx[0][1] + conf_mtrx[0][3] + \
        conf_mtrx[1][0] + conf_mtrx[1][1] + conf_mtrx[1][3] + \
        conf_mtrx[3][0] + conf_mtrx[3][1] + conf_mtrx[3][3])
G_fp = (conf_mtrx[2][0] + conf_mtrx[2][1] + conf_mtrx[2][3])
G_fn = (conf_mtrx[0][2] + conf_mtrx[1][2] + conf_mtrx[3][2])

G_tpr = G_tp / (G_tp + G_fn)
G_tnr = G_tn / (G_tn + G_fp)
G_fpr = G_fp / (G_fp + G_tn)
G_fnr = G_fn / (G_fn + G_tp)

print("Glaucoma Category: ")
print("-->  True Positives: ", G_tp)
print("-->  True Negatives: ", G_tn)
print("-->  False Positives: ", G_fp)
print("-->  False Negatives: ", G_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(G_tpr))
print("-->  True Negative Rate: {0:.4f}".format(G_tnr))
print("-->  False Positive Rate: {0:.4f}".format(G_fpr))
print("-->  False Negative Rate: {0:.4f}".format(G_fnr))

print()
print('=*=*'*8)
print()

# Myopia Evaluation
M_tp = conf_mtrx[3][3]
M_tn = (conf_mtrx[0][0] + conf_mtrx[0][1] + conf_mtrx[0][2] + \
        conf_mtrx[1][0] + conf_mtrx[1][1] + conf_mtrx[1][2] + \
        conf_mtrx[2][0] + conf_mtrx[2][1] + conf_mtrx[2][2])
M_fp = (conf_mtrx[3][0] + conf_mtrx[3][1] + conf_mtrx[3][2])
M_fn = (conf_mtrx[0][3] + conf_mtrx[1][3] + conf_mtrx[2][3])

M_tpr = M_tp / (M_tp + M_fn)
M_tnr = M_tn / (M_tn + M_fp)
M_fpr = M_fp / (M_fp + M_tn)
M_fnr = M_fn / (M_fn + M_tp)

print("Myopia Category: ")
print("-->  True Positives: ", M_tp)
print("-->  True Negatives: ", M_tn)
print("-->  False Positives: ", M_fp)
print("-->  False Negatives: ", M_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(M_tpr))
print("-->  True Negative Rate: {0:.4f}".format(M_tnr))
print("-->  False Positive Rate: {0:.4f}".format(M_fpr))
print("-->  False Negative Rate: {0:.4f}".format(M_fnr))


In [ ]:
def confusion_metrics(cm):
#     TP = cm[1][0]
#     TN = cm[0][1]
#     FP = cm[0][0]
#     FN = cm[1][1]

    TP = (N_tp + C_tp + G_tp + M_tp)
    TN = (N_tn + C_tn + G_tn + M_tn)
    FP = (N_fp + C_fp + G_fp + M_fp)
    FN = (N_fn + C_fn + G_fn + M_fn)
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    
    print("True Positives: ", TP)
    print("True Negatives: ", TN)
    print("False Positives: ", FP)
    print("False Negatives: ", FN)
    print('-'*35)
    print("True Positive Rate: {0:.4f}".format(TPR))
    print("True Negative Rate: {0:.4f}".format(TNR))
    print("False Positive Rate: {0:.4f}".format(FPR))
    print("False Negative Rate: {0:.4f}".format(FNR))
    
    # calculate accuracy
    conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    
    # calculate specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))

    # calculate f1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    
    print('-'*50)
    print(f"Accuracy: {round(conf_accuracy, 4)}")
    print(f"Mis-Classification: {round(conf_misclassification, 4)}")
    print(f"Sensitivity: {round(conf_sensitivity, 4)}")
    print(f"Specificity: {round(conf_specificity,4)}")
    print(f"G-Mean: {round(np.sqrt(round(conf_sensitivity,4) * round(conf_specificity, 4)), 4)}")
    print(f"Precision: {round(conf_precision, 4)}")
    print(f"F1 Score: {round(conf_f1, 4)}")
    
confusion_metrics(cm)

**Sensitivity (True Positive Rate)**

In [ ]:
from sklearn import metrics

# y_pred_raw = model.predict_generator(test_generator, verbose=1)
y_pred_raw = model.predict(x_test1, verbose=1)
y_pred_proba = np.amax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

pred_df = pd.DataFrame({
#     'true_values': test_generator.classes,
    'true_values': y_test_classes,
    'pred_probs': y_pred_proba
})

plt.figure(figsize=(10, 7))

thresholds = np.linspace(0, 1, 200)

def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return (true_positive / (true_positive + false_negative))

def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))

tpr_values = [TPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

plt.plot(fpr_values, # False Positive Rate on x-axis
         tpr_values, # True Positive Rate on y-axis
         label='ROC Curve')

plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')


# plt.title(f"ROC Curve with AUC = {round(metrics.roc_auc_score(pred_df['true_values'], pred_df['pred_probs']), 3)}", fontsize=20)
plt.xlabel("False Positive Rate", fontsize=16)
plt.ylabel("True Positive Rate", fontsize=16)

plt.legend(fontsize=14)

**Specificity Plot**

In [ ]:
# y_pred_raw = model.predict_generator(test_generator, verbose=1)
y_pred_raw = model.predict(x_test1, verbose=1)
y_pred_proba = np.amax(y_pred_raw, axis=1)
y_test_classes = np.argmax(y_test1, axis=1)
pred_df = pd.DataFrame({
    'true_values': y_test_classes,
    'pred_probs': y_pred_proba
})

thresholds = np.linspace(0, 1, 200)

def TNR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return (true_negative / (true_negative + false_positive))

def FNR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return 1 - (true_positive / (true_positive + false_negative))

tnr_values = [TNR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]
fnr_values = [FNR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

plt.plot(fnr_values, # False Positive Rate on X-axis
         tnr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# plt.title(f"ROC Curve with AUC = {round(metrics.roc_auc_score(pred_df['true_values'], pred_df['pred_probs']), 3)}", fontsize=20)
plt.ylabel('True Negative Rate', fontsize=16)
plt.xlabel('False Negative Rate', fontsize=16)

plt.legend(fontsize=14)

**Kappa Score**

In [ ]:
from sklearn import metrics
from matplotlib.cbook import flatten

th = 0.5
# gt = (test_generator.classes)
# gt = (np.argmax(y_test, axis=1))
gt = np.argmax(y_test1, axis=1)
pr = np.argmax(y_pred_raw, axis=1)
kappa = metrics.cohen_kappa_score(gt,pr)
# auc = metrics.roc_auc_score(gt, pr, multi_class='ovr')
f1 = metrics.f1_score(gt, pr, average='micro')
# final_score = (kappa + f1 + auc) / 3.0

print("Evaluation Metrics: ")
print("-> Kappa Score: {0:.4f}".format(kappa))
print("-> F1 Score: {0:.4f}".format(f1))
# print("-> AUC Score: {0:.4f}".format(auc))
# print("-> Final Score: {0:.4f}".format(final_score))

**GRAD-CAM Visualization**

In [ ]:
# from tensorflow.keras import backend as K

# def grad_cam(input_model, image, category_index, layer_name):
#     model = keras.Sequential()
#     model.add(input_model)
    
#     nb_classes = 2
#     target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
#     model.add(layers.Lambda(target_layer, output_shape=target_category_loss_output_shape))
    
#     loss = K.sum(model.layers[-1].output)
#     conv_output = [l for l in model.layers[0].layers if l.name is layer_name][0].output
#     grads = normalize(K.gardients(loss, conv_output)[0])
#     gradient_function = K.function([model.layers[0].input], [conv_output, grads])
    
#     output, grads_val, gradient_function([image])
#     output, grads_val = output[0, :]. gards_val[0, :, :, :]
    
#     weights = np.mean(grads_val, axis=(0,1))
#     cam = np.ones(output.shape[0 : 2], dtype=np.float32)
    
#     for i,w in enumerate(weights):
#         cam += w * output[:, :, i]
    
#     cam = cv2.resize(cam, (256, 256))
#     cam = np.maximum(cam, 0)
#     heatmap = cam / np.max(cam)
    
#     # Return BGR [0...255] from the preprocessed image
#     image = image[0, :]
#     image -= np.min(image)
#     image = np.minimum(image, 255)
    
#     cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
#     cam = np.float32(cam) + np.float32(image)
#     cam = 255 * cam / np.max(cam)
#     return np.uint8(cam), heatmap

# cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
# cv2.imwrite("gradcam.jpg", cam)

# regiter_gradient()
# guided_model = modify_backprop(model, "GuidedBackProp")
# saliency_fn = compile_salienfy_function(guded_model)
# saliency = saliency_fn([preprocessed_input, 0])
# gradcam = saliency[0] * heatmap[..., np.newaxis]
# cv2.imwrite('guided_gradcam.jpg', deprocess_image(gradcam))

In [ ]:
# define layers for producing heat maps

last_conv_layer_name = "conv2d_end"
classifier_layer_names = [
    "global_average_pooling2d",
    "dense_output"
]

In [ ]:
# define heatmap function helpers

def get_img_array(img_path, size):
    # img is a PIL image of size 256x256
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size, color_mode='rgb')
#     img_rgb = tf.keras.preprocessing.image.load_img(img_path, target_size=size, color_mode='rgb')
    
    # array is a float32 Numpy array of shape (256, 256, 1)
    array = tf.keras.preprocessing.image.img_to_array(img)
#     array_rgb = tf.keras.preprocessing.image.img_to_array(img_rgb)
    
    # we add a dimension to transform our array into a "batch"
    # of size (1, 256, 256, 3)
    array = np.expand_dims(array, axis=0)
#     array_rgb = np.expand_dims(array_rgb, axis=0)
    
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # first, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)
    
    # Second, we create a model that maps the actiavtions of the last conv
    # layer to the final class predictions
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)
    
    # Then, we compute the gradient of the top gradient class for our input image
    # with respect to the actiavtions of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # compute class predictions
        preds = classifier_model(last_conv_layer_output)
        
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]
        
    # This is the gradient of the top predicted class with regards to 
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)
    
    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # We multiply each channel in the feature map array
    # by 'how important this channel is' with regards to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
        
    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)
    
    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
# Sort validation set classes into arrays

# total_batch = valid_generator.__len__()
# valid_generator.reset()
# normal_array = []
# cataract_array = []

# for i in range(total_batch):
#     x, y = valid_generator.next()
#     count = 0
#     for _, label in y:
#         if label == 1.0:
#             normal_array.append(x[count])
#         else:
#             cataract_array.append(x[count])
#         count += 1

total_batch = len(x_test) // BATCH_SIZE
normal_array = []
cataract_array = []
glaucoma_array = []
amd_array =[]
myopia_array = []

normal_array1 = []
cataract_array1 = []
glaucoma_array1 = []
amd_array1 =[]
myopia_array1 = []
b = 0

for i in range(total_batch):
  x, y = x_test1[b:b+BATCH_SIZE], y_test1[b:b+BATCH_SIZE]
#   x_1, y_1 = x_test1[b:b+BATCH_SIZE], y_test1[b:b+BATCH_SIZE]
  count = 0
  for j in range(len(y)):
    label = np.argmax(y[j])
    if label == 0:
      normal_array.append(x[count])
#       normal_array1.append(x_1[count])
    elif label == 1:
      cataract_array.append(x[count])
#       cataract_array1.append(x_1[count])
    elif label == 2:
      glaucoma_array.append(x[count])
#       glaucoma_array1.append(x_1[count])
    elif label == 3:
      myopia_array.append(x[count])
#       myopia_array1.append(x_1[count])
    
    count += 1
  b += BATCH_SIZE



In [ ]:
y_test1[7]

In [ ]:
# idx 1 = normal
# idx 2 = cataract
# idx 7 = glaucoma
# idx 4 = myopia

# ['N', 'C', 'G', 'M']

img = keras.preprocessing.image.array_to_img(x_test1[4])
img_sample = np.expand_dims(x_test1[4], axis=0)
# plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.imshow(img)
plt.title(f"{class_labels[np.argmax(model.predict(img_sample))]}")

In [ ]:
sample = np.expand_dims(x_test1[7],axis=0)
inference = model.predict(sample)
print(np.argmax(inference[0]))
print(y_test1[7])

In [ ]:
# Define heatmap actiavtion display in a graph
import matplotlib as mp

def display_activation_graph(image_array, label):
    # display predictions on validation set
    plt.figure(figsize=(10, 10)) # specificying the overall grid size
    correct = 0
    plt.subplots_adjust(wspace=0.01, hspace=0.14)
    plt.rcParams['font.size'] = 12
    for i in range(9):
        plt.subplot(3,3,i+1) # the number of images in the grid is 4*5 (25)
        pil_img = tf.keras.preprocessing.image.array_to_img(image_array[i])
#         plt.imshow(pil_img, cmap='gray', vmin=0, vmax=255)
        plt.imshow(pil_img)
        
        inference = model.predict(np.array([image_array[i],]))
        verdict = np.argmax(inference[0])
        if verdict == label:
            correct += 1
            
        plt.title(f"{label} == {verdict}")
        plt.axis('off')
    plt.suptitle(f"Model Prediction on Validation Set\n {correct}/9 Correct")
    plt.show()
    
    # display heatmap
    plt.figure(figsize=(10, 10)) # speciffying the overall grid size
    correct = 0
    plt.subplots_adjust(wspace=0.01, hspace=0.14)
    plt.rcParams['font.size'] = 12
    for i in range(9):
        plt.subplot(3, 3, i+1) # the number of images in the grid is 4*5 (25)
#         plt.subplots_adjust(wspace=0.2, hspace=0.2)
        pil_img = tf.keras.preprocessing.image.array_to_img(image_array[i])
        inference = model.predict(np.array([image_array[i],]))
        heatmap = make_gradcam_heatmap(np.array([image_array[i],]),
                                          model,
                                          last_conv_layer_name,
                                          classifier_layer_names
                                      )
        img = image_array[i]
        heatmap = np.uint8(255 * heatmap)
        jet = mp.cm.get_cmap("jet")
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]
        jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)
        superimposed_img = jet_heatmap * 0.4 + img
        superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
        plt.imshow(superimposed_img)
        verdict = np.argmax(inference[0])
        if verdict == label:
            correct += 1
            
        plt.title(f"{label} == {verdict}")
        plt.axis('off')
    plt.suptitle(f"Heat Map Representation \n {correct}/9 Correct")
    plt.show()
    
    # display class activation 
    plt.figure(figsize=(10, 10)) # specifying the overall grid size
    plt.subplots_adjust(wspace=0.01, hspace=0.14)
    plt.rcParams['font.size'] = 12
    correct = 0
    for i in range(9):
        plt.subplot(3,3, i+1) # the number of images in the grid is 4*5 (25)
        pil_img = tf.keras.preprocessing.image.array_to_img(image_array[i])
        inference = model.predict(np.array([image_array[i],]))
        heatmap = make_gradcam_heatmap(np.array([image_array[i],]),
                                       model,
                                       last_conv_layer_name,
                                       classifier_layer_names
                                      )
        plt.imshow(heatmap)
        verdict = np.argmax(inference[0])
        if verdict == label:
            correct += 1
        plt.title(f"{label} == {verdict}")
        plt.axis('off')
    plt.suptitle(f"Class Activation")
    plt.show()

In [ ]:
display_activation_graph(normal_array, 0)

In [ ]:
display_activation_graph(cataract_array, 1)

In [ ]:
display_activation_graph(glaucoma_array, 2)


In [ ]:
display_activation_graph(myopia_array, 3)

In [ ]:
# display_activation_graph(myopia_array, 4)

# Step 8: Test the Model

In [ ]:
X5[rand_idx].shape

In [ ]:
train_output = "preprocess_train_gray"
class_labels = ['N', 'C','G','M']
rand_idx = np.random.randint(0, len(x_test))
X_min = x_test1[300:]
# img_file = os.path.join(train_output, df_test.iloc[rand_idx, 0])
img_file = x_test1[rand_idx]
image = keras.preprocessing.image.array_to_img(img_file)
# image = keras.preprocessing.image.load_img(img_file, color_mode='grayscale')
x = keras.preprocessing.image.img_to_array(image)
# x = x * 1./255
x = np.expand_dims(x, axis=0)
pred = model.predict(x)
print(pred)

plt.imshow(image)
# plt.title("Original: " + df_test.iloc[rand_idx, 1])
plt.title("Original: " + class_labels[np.argmax(y_test1[rand_idx])])
plt.xlabel("Prediction: " + class_labels[np.argmax(pred[0])])
plt.grid(False)
# plt.axis('off')
plt.show()

In [ ]:
import re
from tensorflow.keras.preprocessing import image

str_classes = ['Normal', 'Cataract', 'Glaucoma', 'Myopia']

def draw_test(name, pred, im, true_label):
  BLACK = [0, 0, 0]
  expanded_image = cv2.copyMakeBorder(im, 0, 0, 10, 280, cv2.BORDER_CONSTANT, value=BLACK)
  cv2.putText(expanded_image, "predicted: " + pred, (280, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (51, 171, 249), 2)
  cv2.putText(expanded_image, "true: " + true_label, (280, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 255, 0), 2)
  return expanded_image

# def getRandomImage(path, img_size=(256, 256)):
#   # loads a random image
#   rand_idx = np.random.randint(0, len(df_test))
#   rand_file = df_test.iloc[rand_idx, 0]
#   label = df_test.iloc[rand_idx, 1]
#   label = str_classes[np.argmax(y_test[rand_idx])]
#   image_path = os.path.join(path, rand_file)
#   return image.load_img(image_path, target_size=img_size, color_mode='grayscale'), \
#           image_path, label, rand_file

def getRandomImage(img_size=(256, 256)):
  # loads random image
  rand_idx = np.random.randint(0, len(x_test1))
  rand_file = x_test[rand_idx]
#   rand_file = df_test.iloc[rand_idx, 0]
  img = x_test1[rand_idx]
  label = str_classes[np.argmax(y_test1[rand_idx])]
  img = keras.preprocessing.image.array_to_img(img)
  return img, label, rand_idx, rand_file


# we use a very low learning rate
fig = plt.figure(figsize=(16, 16))
axes = []
files = []
predictions = []
true_labels = []
rand_files = []
rand_idxs = []
rows = 9
cols = 3

# predicting the images

for i in range(9):
  # path = "preprocess_train_gray"
  path = prep_train
#   img, final_path, true_label, rand_file = getRandomImage(path)
  img, true_label, rand_idx, rand_file = getRandomImage()
#   files.append(final_path)
  rand_files.append(rand_file)
  rand_idxs.append(rand_idx)
  true_labels.append(true_label)
#   x = keras.preprocessing.image.img_to_array(img)
  x = x_test1[rand_idx]
#   x = img
#   x = x * 1./255
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  predictions.append(classes)

plt.rcParams['font.size'] = 6

temp_path = "temp_img"

if os.path.exists(temp_path):
  clear_content(temp_path)
else:
  os.mkdir(temp_path)

for i in range(len(rand_idxs)):
  rgb_path = prep_train
#   if rand_files[i] in os.listdir(prep_train):
#         imgpath = os.path.join(rgb_path, rand_files[i])
#   image = cv2.imread(files[i])
  # image = cv2.imread(imgpath)
  image = keras.preprocessing.image.array_to_img(x_test1[rand_idxs[i]])
#   prep_img = cv2.imread(os.path.join(prep_train, df_test.iloc[rand_idxs[i], 0]))
  image = cv2.imwrite(os.path.join(temp_path, f"fundus_{rand_idxs[i]}.jpg"), x_test1[rand_idxs[i]])
#   image = cv2.imwrite(os.path.join(temp_path, f"fundus_{rand_idxs[i]}.jpg"), image)
  image = cv2.imread(os.path.join(temp_path, f"fundus_{rand_idxs[i]}.jpg"))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = draw_test("Prediction", str_classes[np.argmax(predictions[i])], image, true_labels[i])
  axes.append(fig.add_subplot(rows, cols, i+1))
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.title("{0}: {1:.2f}%".format(str_classes[np.argmax(predictions[i])], ((np.amax(predictions[i], 1)) * 100)[0]))
  plt.grid(False)
  plt.axis('off')


plt.show()

In [ ]:
def round_up(n, decimals=0):
  multiplier = 10 ** decimals
  return np.ceil(n * multiplier) / multiplier

In [ ]:
from matplotlib import pyplot as plt
folders = ['normal', 'cataract','glaucoma','myopia']
test_folder = "datazet/datazet"
n_count = 0
i=0

plt.rcParams['figure.figsize'] = (25.0, 15.0)
plt.rcParams.update({'font.size': 12})
plt.subplots_adjust(wspace=0.1, hspace=0.15)

direc_path = os.path.join(test_folder, folders[0])
for file in os.listdir(direc_path):
    file_path = os.path.join(direc_path, file)
    image = keras.preprocessing.image.load_img(file_path, color_mode="rgb", target_size=(IMG_SIZE, IMG_SIZE))
    x = keras.preprocessing.image.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
#         print(pred)
    prediction_class = folders[np.argmax(pred[0])]
    true_class = folders[0]

    plt.subplot(5, 10, i+1)
    plt.imshow(image)
    plt.title(f"True: {true_class}")
    plt.xlabel(f"Prediction: {prediction_class}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    ax = plt.gca()
#         ax.get_xaxis().set_visible(False)
    if true_class == prediction_class:
        n_count+=1
    i+=1
plt.suptitle(f"Normal Prediction on Test Images\n {n_count}/50 Correct")
plt.show()
            

In [ ]:
plt.rcParams['figure.figsize'] = (25.0, 15.0)
plt.rcParams.update({'font.size': 12})
plt.subplots_adjust(wspace=0.1, hspace=0.15)
i=0
c_count = 0
direc_path = os.path.join(test_folder, folders[1])
for file in os.listdir(direc_path):
    file_path = os.path.join(direc_path, file)
    image = keras.preprocessing.image.load_img(file_path, color_mode="rgb", target_size=(IMG_SIZE, IMG_SIZE))
    x = keras.preprocessing.image.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
#         print(pred)
    prediction_class = folders[np.argmax(pred[0])]
    true_class = folders[1]

    plt.subplot(5, 10, i+1)
    plt.imshow(image)
    plt.title(f"True: {true_class}")
    plt.xlabel(f"Prediction: {prediction_class}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    ax = plt.gca()
#         ax.get_xaxis().set_visible(False)
    if true_class == prediction_class:
        c_count+=1
    i+=1
plt.suptitle(f"Cataract Prediction on Test Images\n {c_count}/50 Correct")
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (25.0, 15.0)
plt.rcParams.update({'font.size': 12})
plt.subplots_adjust(wspace=0.1, hspace=0.15)
i=0
g_count =0
direc_path = os.path.join(test_folder, folders[2])
for file in os.listdir(direc_path):
    file_path = os.path.join(direc_path, file)
    image = keras.preprocessing.image.load_img(file_path, color_mode="rgb", target_size=(IMG_SIZE, IMG_SIZE))
    x = keras.preprocessing.image.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
#         print(pred)
    prediction_class = folders[np.argmax(pred[0])]
    true_class = folders[2]

    plt.subplot(5, 10, i+1)
    plt.imshow(image)
    plt.title(f"True: {true_class}")
    plt.xlabel(f"Prediction: {prediction_class}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    ax = plt.gca()
#         ax.get_xaxis().set_visible(False)
    if true_class == prediction_class:
        g_count+=1
    i+=1
plt.suptitle(f"Glaucoma Prediction on Test Images\n {g_count}/50 Correct")
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (25.0, 15.0)
plt.rcParams.update({'font.size': 12})
plt.subplots_adjust(wspace=0.1, hspace=0.15)

i=0
m_count = 0

direc_path = os.path.join(test_folder, folders[3])
for file in os.listdir(direc_path):
    file_path = os.path.join(direc_path, file)
    image = keras.preprocessing.image.load_img(file_path, color_mode="rgb", target_size=(IMG_SIZE, IMG_SIZE))
    x = keras.preprocessing.image.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
#         print(pred)
    prediction_class = folders[np.argmax(pred[0])]
    true_class = folders[3]

    plt.subplot(5, 10, i+1)
    plt.imshow(image)
    plt.title(f"True: {true_class}")
    plt.xlabel(f"Prediction: {prediction_class}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    ax = plt.gca()
#         ax.get_xaxis().set_visible(False)
    if true_class == prediction_class:
        m_count+=1
    i+=1
plt.suptitle(f"Myopia Prediction on Test Images\n {m_count}/50 Correct")
plt.show()

In [ ]:
from matplotlib import pyplot as plt
folders = ['normal', 'cataract','glaucoma','myopia']
test_folder = "datazet/datazet"
n_count = 0
c_count = 0
m_count = 0
g_count = 0
i=0
used_indices = []

plt.rcParams['figure.figsize'] = (25.0, 60.0)
plt.rcParams.update({'font.size': 10})
plt.subplots_adjust(wspace=0.1, hspace=0.3)

for n in range(4):
    direc_path = os.path.join(test_folder, folders[n])
    for file in os.listdir(direc_path):
        file_path = os.path.join(direc_path, file)
        image = keras.preprocessing.image.load_img(file_path, color_mode="rgb", target_size=(IMG_SIZE, IMG_SIZE))
        x = keras.preprocessing.image.img_to_array(image)
        x = np.expand_dims(x, axis=0)
        pred = model.predict(x)
#         print(pred)
        prediction_class = folders[np.argmax(pred[0])]
        true_class = folders[n]
        
        plt.subplot(20, 10, i+1)
        plt.imshow(image)
        plt.title(f"True: {true_class}")
        plt.xlabel(f"Prediction: {prediction_class}")
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        ax = plt.gca()
#         ax.get_xaxis().set_visible(False)
        if folders[n] == prediction_class:
            if folders[n] == 'normal':
                n_count += 1
            elif folders[n] == 'cataract':
                c_count += 1
            elif folders[n] == 'myopia':
                m_count += 1
            elif folders[n] == 'glaucoma':
                g_count += 1
        i+=1
    if folders[n] == 'normal':
        plt.suptitle(f"Model Prediction on Test Images\n {n_count}/50 Correct")
    elif folders[n] == 'cataract':
        plt.suptitle(f"Model Prediction on Test Images\n {c_count}/50 Correct")
    elif folders[n] == 'glaucoma':
        plt.suptitle(f"Model Prediction on Test Images\n {g_count}/50 Correct")
    elif folders[n] == 'myopia':
        plt.suptitle(f"Model Prediction on Test Images\n {m_count}/50 Correct")
    
plt.show()
print("Test Summary:")
print('->Normal: {0}/50'.format(n_count))
print('->Cataract: {0}/50'.format(c_count))
print('->Myopia: {0}/50'.format(m_count))
print('->Glaucoma: {0}/50'.format(g_count))
            

In [ ]:
os.listdir(os.path.join(test_folder, folders[0]))

In [ ]:
# from zipfile import ZipFile
# from tqdm import tqdm

# zipped = "odir_samples.zip"
# direc = "temp_img"
# preds = "preds"

# if os.path.exists(os.path.join(direc, preds)):
#   clear_content(os.path.join(direc, preds))
# else:
#   os.mkdir(os.path.join(direc, preds))

# df = pd.DataFrame({
#     'labels': true_labels,
#     'predictions': [np.argmax(pred) for pred in predictions]    
# })

# df.to_csv(os.path.join(direc, preds, "predictions.csv"))

# with ZipFile(zipped, "w") as zipObj:
#   for dirname, subfolders, filenames in os.walk(direc):
#     for file in tqdm(filenames):
#       zipObj.write(os.path.join(dirname, file))

# print()
# print(f"{zipped} created!")

In [ ]:
# FileLink("odir_samples.zip")

#### Convert TF model to TFLite and SavedModel

In [ ]:
plt.rcParams['figure.figsize'] = (25.0, 15.0)
plt.rcParams.update({'font.size': 12})
plt.subplots_adjust(wspace=0.1, hspace=0.15)
i=0
g_count =0

for file in tqdm(glaucoma_prepped):
    image = keras.preprocessing.image.array_to_img(file)
    x = keras.preprocessing.image.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
#         print(pred)
    prediction_class = folders[np.argmax(pred[0])]
    true_class = folders[2]

    plt.subplot(5, 10, i+1)
    plt.imshow(image)
    plt.title(f"True: {true_class}")
    plt.xlabel(f"Prediction: {prediction_class}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    ax = plt.gca()
#         ax.get_xaxis().set_visible(False)
    if true_class == prediction_class:
        g_count+=1
    i+=1
plt.suptitle(f"Glaucoma Prediction on Test Images\n {g_count}/40 Correct")
plt.show()